# Chương 15: Cải Thiện Hiệu Suất với Ensembles

Các phương pháp ensemble có thể giúp cải thiện độ chính xác cho tập dữ liệu của bạn. Trong chương này, bạn sẽ khám phá các phương pháp tạo ra một số kiểu ensemble mạnh mẽ nhất trong Python với scikit-learn. Bài học này sẽ hướng dẫn bạn về Boosting, Bagging và Majority Voting, giúp bạn tiếp tục nâng cao độ chính xác của các mô hình trên tập dữ liệu riêng. Sau khi hoàn thành bài học này, bạn sẽ biết:

1. Cách sử dụng các phương pháp ensemble bagging như cây quyết định bagging, rừng ngẫu nhiên và extra trees.
2. Cách sử dụng các phương pháp ensemble boosting như AdaBoost và stochastic gradient boosting.
3. Cách sử dụng các phương pháp ensemble voting để kết hợp dự đoán từ nhiều thuật toán khác nhau.

## 15.1 Kết Hợp Các Mô Hình Thành Dự Đoán Ensemble

Ba phương pháp phổ biến nhất để kết hợp dự đoán từ các mô hình khác nhau là:

- **Bagging**: Xây dựng nhiều mô hình (thường cùng loại) từ các mẫu con khác nhau của tập dữ liệu huấn luyện.
- **Boosting**: Xây dựng nhiều mô hình (thường cùng loại), mỗi mô hình học cách sửa lỗi dự đoán của mô hình trước đó trong chuỗi các mô hình.
- **Voting**: Xây dựng nhiều mô hình (thường khác loại) và sử dụng các phép thống kê đơn giản (như tính trung bình) để kết hợp các dự đoán.

Phần này giả định bạn đã quen thuộc với các thuật toán học máy và phương pháp ensemble và sẽ không đi sâu vào chi tiết cách thức hoạt động của các thuật toán hoặc tham số của chúng.

Tập dữ liệu Pima Indians Diabetes được sử dụng để minh họa cho từng thuật toán. Mỗi thuật toán ensemble được minh họa bằng cách sử dụng cross validation 10-fold và độ chính xác phân loại làm chỉ số đánh giá hiệu suất.

## 15.2 Thuật Toán Bagging

**Bootstrap Aggregation** (hay Bagging) liên quan đến việc lấy nhiều mẫu từ tập dữ liệu huấn luyện của bạn (có thay thế) và huấn luyện một mô hình cho mỗi mẫu. Dự đoán đầu ra cuối cùng là trung bình của các dự đoán từ tất cả các mô hình con. Ba mô hình bagging được đề cập trong phần này bao gồm:

- Bagged Decision Trees (Cây quyết định bagging)
- Random Forest (Rừng ngẫu nhiên)
- Extra Trees (Cây cực ngẫu nhiên)

### 15.2.1 Bagged Decision Trees

Bagging hoạt động tốt nhất với các thuật toán có phương sai cao. Một ví dụ phổ biến là cây quyết định, thường được xây dựng mà không cắt tỉa. Trong ví dụ dưới đây là cách sử dụng `BaggingClassifier` với thuật toán Classification and Regression Trees (DecisionTreeClassifier). Tổng cộng 100 cây được tạo ra.

In [ ]:
# Bagged Decision Trees for Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
kfold = KFold(n_splits=10, random_state=seed)
cart = DecisionTreeClassifier()
num_trees = 100
model = BaggingClassifier(base_estimator=cart, n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

### 15.2.2 Random Forest

Random Forest là một phần mở rộng của cây quyết định bagging. Các mẫu của tập dữ liệu huấn luyện được lấy có thay thế, nhưng các cây được xây dựng theo cách giảm sự tương quan giữa các bộ phân loại riêng lẻ. Cụ thể, thay vì tham lam chọn điểm phân chia tốt nhất trong quá trình xây dựng mỗi cây, chỉ một tập con ngẫu nhiên các đặc trưng được xem xét cho mỗi lần phân chia. Bạn có thể xây dựng mô hình Random Forest cho phân loại bằng lớp `RandomForestClassifier`. Ví dụ dưới đây minh họa việc sử dụng Random Forest cho phân loại với 100 cây và các điểm phân chia được chọn từ tập hợp ngẫu nhiên gồm 3 đặc trưng.

In [ ]:
# Random Forest Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_trees = 100
max_features = 3
kfold = KFold(n_splits=10, random_state=7)
model = RandomForestClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

### 15.2.3 Extra Trees

Extra Trees là một biến thể khác của bagging, trong đó các cây ngẫu nhiên được xây dựng từ các mẫu của tập dữ liệu huấn luyện. Bạn có thể xây dựng mô hình Extra Trees cho phân loại bằng lớp `ExtraTreesClassifier`. Ví dụ dưới đây minh họa Extra Trees với số lượng cây được đặt là 100 và các điểm phân chia được chọn từ 7 đặc trưng ngẫu nhiên.

In [ ]:
# Extra Trees Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import ExtraTreesClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_trees = 100
max_features = 7
kfold = KFold(n_splits=10, random_state=7)
model = ExtraTreesClassifier(n_estimators=num_trees, max_features=max_features)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

## 15.3 Thuật Toán Boosting

**Thuật toán ensemble Boosting** tạo ra một chuỗi các mô hình cố gắng sửa chữa lỗi của các mô hình đứng trước chúng trong chuỗi. Sau khi được tạo ra, các mô hình đưa ra dự đoán có thể được đánh trọng số theo độ chính xác đã chứng minh của chúng và kết quả được kết hợp để tạo ra dự đoán đầu ra cuối cùng. Hai thuật toán ensemble boosting phổ biến nhất là:

- AdaBoost
- Stochastic Gradient Boosting

### 15.3.1 AdaBoost

AdaBoost có lẽ là thuật toán ensemble boosting thành công đầu tiên. Nhìn chung, nó hoạt động bằng cách đánh trọng số các trường hợp trong tập dữ liệu dựa trên mức độ dễ hay khó phân loại, cho phép thuật toán chú ý hơn hoặc ít hơn đến chúng trong quá trình xây dựng các mô hình tiếp theo. Bạn có thể xây dựng mô hình AdaBoost cho phân loại bằng lớp `AdaBoostClassifier`. Ví dụ dưới đây minh họa việc xây dựng 30 cây quyết định liên tiếp bằng thuật toán AdaBoost.

In [ ]:
# AdaBoost Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
num_trees = 30
seed=7
kfold = KFold(n_splits=10, random_state=seed)
model = AdaBoostClassifier(n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

### 15.3.2 Stochastic Gradient Boosting

Stochastic Gradient Boosting (còn được gọi là Gradient Boosting Machines) là một trong những kỹ thuật ensemble tinh vi nhất. Đây cũng là một kỹ thuật đang chứng minh có lẽ là một trong những kỹ thuật tốt nhất hiện có để cải thiện hiệu suất thông qua ensembles. Bạn có thể xây dựng mô hình Gradient Boosting cho phân loại bằng lớp `GradientBoostingClassifier`. Ví dụ dưới đây minh họa Stochastic Gradient Boosting cho phân loại với 100 cây.

In [ ]:
# Stochastic Gradient Boosting Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
seed = 7
num_trees = 100
kfold = KFold(n_splits=10, random_state=seed)
model = GradientBoostingClassifier(n_estimators=num_trees, random_state=seed)
results = cross_val_score(model, X, Y, cv=kfold)
print(results.mean())

## 15.4 Voting Ensemble

**Voting** là một trong những cách đơn giản nhất để kết hợp dự đoán từ nhiều thuật toán học máy. Nó hoạt động bằng cách đầu tiên tạo ra hai hoặc nhiều mô hình độc lập từ tập dữ liệu huấn luyện của bạn. Sau đó, một Voting Classifier có thể được sử dụng để đóng gói các mô hình của bạn và tính trung bình các dự đoán của các mô hình con khi được yêu cầu đưa ra dự đoán cho dữ liệu mới. Các dự đoán của các mô hình con có thể được đánh trọng số, nhưng việc chỉ định trọng số cho các bộ phân loại một cách thủ công hoặc thậm chí theo trực giác là khó khăn.

Các phương pháp tiên tiến hơn có thể học cách đánh trọng số tốt nhất cho các dự đoán từ các mô hình con, nhưng phương pháp này được gọi là stacking (stacked aggregation) và hiện không được cung cấp trong scikit-learn.

Bạn có thể tạo mô hình ensemble voting cho phân loại bằng lớp `VotingClassifier`. Đoạn mã dưới đây cung cấp một ví dụ về việc kết hợp các dự đoán của hồi quy logistic, cây phân loại và hồi quy, và máy vector hỗ trợ cho một bài toán phân loại.

In [ ]:
# Voting Ensemble for Classification
from pandas import read_csv
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier

filename = 'pima-indians-diabetes.data.csv'
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = read_csv(filename, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
kfold = KFold(n_splits=10, random_state=7)
# create the sub models
estimators = []
model1 = LogisticRegression()
estimators.append(('logistic', model1))
model2 = DecisionTreeClassifier()
estimators.append(('cart', model2))
model3 = SVC()
estimators.append(('svm', model3))
# create the ensemble model
ensemble = VotingClassifier(estimators)
results = cross_val_score(ensemble, X, Y, cv=kfold)
print(results.mean())

## 15.5 Tóm Tắt

Trong chương này, bạn đã khám phá các thuật toán ensemble học máy để cải thiện hiệu suất của các mô hình trên các bài toán của bạn. Bạn đã học về:

- **Bagging Ensembles** bao gồm Bagged Decision Trees, Random Forest và Extra Trees.
- **Boosting Ensembles** bao gồm AdaBoost và Stochastic Gradient Boosting.
- **Voting Ensembles** để lấy trung bình các dự đoán cho bất kỳ mô hình tùy ý nào.